In [1]:
import numpy as np
from random import randint
import math

### UTILS

In [2]:
def get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS):
    return {
        "NUM_KNAPSACKS": NUM_KNAPSACKS,
        "NUM_ITEMS": NUM_ITEMS,
        "NUM_DIMENSIONS": NUM_DIMENSIONS,
        "VALUES": VALUES,
        "WEIGHTS": WEIGHTS,
        "CONSTRAINTS": CONSTRAINTS,
    }

In [3]:
def generate_random_solution(num_knapsacks, num_items):
    """
    generate a pure random solution. In this case is possible to start
    from a starting point where you have items that are carried by more than 1 knapsack.
    """
    
    solution = np.array(
        [np.random.random(num_items) < 0.5 for _ in range(num_knapsacks)], dtype=np.int8
    )

    return solution

In [4]:
def generate_clean_random_solution(num_knapsacks, num_items):
    """
    generate a random solution that satisfy the first constraint
    (that is each item is either not carried or carried from exactly 1 item).
    """

    # generate a 2D matrix (num_knapsacks, num_items) full of 0s.
    solution = np.zeros((num_knapsacks, num_items), dtype=int)

    # iterate over the columns
    for item_idx in range(num_items):

        # generate random number:
        # if 0 the item is not carried
        # if 1 the item is carried by 1 knapsack
        carried = randint(0, 1)
    
        if carried == 0:
            continue
        
        # generate a random index that identifies the knapsack that carries the item
        id_knapsack = randint(0, num_knapsacks - 1)

        # update the solution
        solution[id_knapsack, item_idx] = 1
        
    return solution

In [5]:
def fitness(solution, problem):
    """
    Calculates the fitness of a solution. It is defined by the overall valued
    carried by all knapsacks.
    """

    # check the basic constraint: an item cannot be carried by more then 1 knapsack
    # if the sum of the items in the same column >= 1 it means that
    # at leats 2 knapsacks are carrying the item, so a fitness of 0 is returned
    if np.any(np.sum(solution, axis=0) > 1):
        return 0

    values = problem["VALUES"]
    weights = problem["WEIGHTS"]
    constraints = problem["CONSTRAINTS"]

    # sum all the values for each row based on the items that each knapsack carries
    total_value = np.sum(solution * values)
    # get the total weights for each knapsack
    knapsack_loads = solution @ weights

    # Check if any constraint is violated
    if np.any(knapsack_loads > constraints):
        return 0 
    else:
        return total_value

In [6]:
def tweak(solution, problem):
    """
    Creates a "neighbor" solution by making one small, random change.
    It randomly picks one item and changes its knapsack assignment.
    """
    # get problem parameters
    num_items = problem["NUM_ITEMS"]
    num_knapsacks = problem["NUM_KNAPSACKS"]
    
    # create a copy to avoid modifying the original solution
    neighbor_solution = solution.copy()
    
    # pick a random item to move
    item_to_move = np.random.randint(num_items)
    # pick a random new knapsack for it (-1 means removing it)
    new_knapsack = np.random.randint(-1, num_knapsacks)
    
    # first, remove the item from its old position (if any)
    neighbor_solution[:, item_to_move] = 0
    # if the item is being assigned to a new knapsack (not just removed)
    if new_knapsack != -1:
        # place the item in the new knapsack
        neighbor_solution[new_knapsack, item_to_move] = 1
        
    return neighbor_solution

### ALGORITHMS

A new version of hill climbing has been implemented
This version of hill climbing solves 1 constraint at the time. It is organized in 2 phases.
- PHASE 1: It first make sure that the solution satisfies a specific constraint, then we keep iterating the process checking the solution for all the other constraint.
- PHASE 2: The hill climbing algorithm is applied an iteratively looks for a slightly better solution by making a tweak of only 1 element.

In [7]:
def guided_hill_climbing(problem, max_steps=10000):
    """
    This version of hill climbing focuses on 1 constraint at the time.
    It first repairs the solution to make it valid, then optimizes it for value.
    """
    num_knapsacks = problem["NUM_KNAPSACKS"]
    num_items = problem["NUM_ITEMS"]
    weights = problem["WEIGHTS"]
    values = problem["VALUES"]
    constraints = problem["CONSTRAINTS"]
    
    # start with a random solution
    current_solution = generate_clean_random_solution(num_knapsacks, num_items)
    
    # PHASE 1
    # taking an invalid solution and repair it
    print("Starting Repair Phase...")
    
    # loop until the solution becomes valid
    while True:
        current_loads = current_solution @ weights
        overload = current_loads - constraints
        overweight_knapsacks = np.where(np.any(overload > 0, axis=1))[0]
        
        if len(overweight_knapsacks) == 0:
            print("Solution is now valid. Moving to Optimization Phase.")
            break 
            
        knapsack_to_fix = overweight_knapsacks[0]
        items_in_knapsack = np.where(current_solution[knapsack_to_fix] == 1)[0]
        
        if len(items_in_knapsack) == 0:
            continue

        item_to_remove = np.random.choice(items_in_knapsack)
        current_solution[knapsack_to_fix, item_to_remove] = 0
        
    # PHASE 2
    # now the solution is valid, so we optimize it
    current_fitness = np.sum(current_solution * values)
    print(f"Initial valid fitness: {current_fitness}")
    
    # start the optimization loop for a fixed number of steps
    for i in range(max_steps):
        # *** REFACTORED PART ***
        # create a neighbor solution by calling the new tweak function
        neighbor_solution = tweak(current_solution, problem)
        
        # calculate the loads of this new neighbor solution
        neighbor_loads = neighbor_solution @ weights
        # check if the new solution violates any constraints
        if not np.any(neighbor_loads > constraints):
            # if it's valid, calculate its fitness
            neighbor_fitness = np.sum(neighbor_solution * values)

            # check if the neighbor's fitness is better than our current best
            if neighbor_fitness > current_fitness:
                current_solution = neighbor_solution
                current_fitness = neighbor_fitness
                print(f"Step {i}: Found better solution with fitness {current_fitness}")

    final_fitness = fitness(current_solution, problem)
    return current_solution, final_fitness

### TEST PROBLEMS

In [8]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# getting problem p1
p1 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

In [9]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

p2 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

In [10]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

p3 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

### FINAL TEST - GUIDED HILL CLIMBING algorithm

In [11]:
print("PROBLEM 1")
best_solution, best_fitness = guided_hill_climbing(p1)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 1
Starting Repair Phase...
Solution is now valid. Moving to Optimization Phase.
Initial valid fitness: 359
Step 1: Found better solution with fitness 430
Step 3: Found better solution with fitness 482
Step 4: Found better solution with fitness 560
Step 10: Found better solution with fitness 629
Step 11: Found better solution with fitness 714
Step 15: Found better solution with fitness 791
Step 21: Found better solution with fitness 834
Step 146: Found better solution with fitness 917
Best fitness: 917
Best solution:
 [[0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0]]


In [12]:
print("PROBLEM 2")
best_solution, best_fitness = guided_hill_climbing(p2)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 2
Starting Repair Phase...
Solution is now valid. Moving to Optimization Phase.
Initial valid fitness: 22010
Step 10: Found better solution with fitness 22199
Step 13: Found better solution with fitness 22835
Step 15: Found better solution with fitness 23512
Step 19: Found better solution with fitness 23875
Step 27: Found better solution with fitness 24561
Step 32: Found better solution with fitness 24873
Step 35: Found better solution with fitness 25074
Step 46: Found better solution with fitness 25774
Step 56: Found better solution with fitness 26667
Step 68: Found better solution with fitness 27489
Step 71: Found better solution with fitness 27628
Step 73: Found better solution with fitness 28071
Step 114: Found better solution with fitness 28521
Step 136: Found better solution with fitness 28615
Step 140: Found better solution with fitness 29555
Step 162: Found better solution with fitness 30237
Step 209: Found better solution with fitness 30329
Step 212: Found better solut

In [13]:
print("PROBLEM 3")
best_solution, best_fitness = guided_hill_climbing(p3, max_steps=1000)

print(f"Best fitness: {best_fitness}")
# print(f"Best solution:\n {best_solution}")

PROBLEM 3
Starting Repair Phase...
Solution is now valid. Moving to Optimization Phase.
Initial valid fitness: 974779
Step 0: Found better solution with fitness 975121
Step 7: Found better solution with fitness 975428
Step 14: Found better solution with fitness 976269
Step 15: Found better solution with fitness 977045
Step 20: Found better solution with fitness 977697
Step 23: Found better solution with fitness 977783
Step 33: Found better solution with fitness 977999
Step 39: Found better solution with fitness 978939
Step 41: Found better solution with fitness 978942
Step 44: Found better solution with fitness 979501
Step 50: Found better solution with fitness 980083
Step 53: Found better solution with fitness 980388
Step 54: Found better solution with fitness 980870
Step 55: Found better solution with fitness 980966
Step 56: Found better solution with fitness 981852
Step 58: Found better solution with fitness 982361
Step 62: Found better solution with fitness 982669
Step 66: Found be